# Simple Maze Problem
- Implement a simple 2D-maze game in which a single player can move around in the four major directions
- Initialise the maze as a 5 x 5 grid to which the player is confined. One of the 25 grid cells is the "goal" that a player called the *seeker* must reach
- Employ an RL algorithm instead of hard-coding a solution so that the seeker learns to find the goal
- Run simulations of the maze repeatedly, rewarding the seeker for finding the goal and smartly keeping track of which of the seeker's decisions worked and which didn't because running simulations can be parallelised and out RL algorithm can also be trained in parallel, we use the Ray API to parallelise the whole process

In [4]:
import random


class Discrete:
    def __init__(self, num_actions: int):
        """Discrete action space for num_actions. Discrete(4) can be used as encoding moving in one of the cardinal directions."""
        self.n = num_actions
    
    def sample(self):
        return random.randint(0, self.n - 1)

space = Discrete(4)   # 0, 1, 2, 3 corresponds to down, left, up and right
print(space.sample())

2


In [6]:
import os


class Environment:
    def __init__(self, *args, **kwargs):
        self.seeker, self.goal = (0, 0), (4, 4)  # seeker gets initialised in the top left, the goal in the bottom right of the maze
        self.info = {'seeker': self.seeker, 'goal': self.goal}

        self.action_space = Discrete(4)  # our seeker can move down, left, up and right
        self.observation_space = Discrete(5*5)  # it can be in a total of 25 states, one for each position on the grid

    def reset(self):  # to play a new game, reset the grid to its original state
        """Reset seeker position and return observations"""
        self.seeker = (0, 0)
        return self.get_observation()
    
    def get_observation(self):
        """Encode the seeker position as integer"""
        return 5 * self.seeker[0] + self.seeker[1]  # convert the seeker tuple to a value from the environment's observation space
    
    def get_reward(self):
        """Reward finding the goal"""
        return 1 if self.seeker == self.goal else 0  # the seeker is rewardded only upon reaching the goal
    
    def is_done(self):
        """We're done if we found the goal"""
        return self.seeker == self.goal  # if the seeker is at the goal, the game is over
    
    def step(self, action):
        """Take a step in a direction and  return all available information"""
        if action == 0:  # move down
            self.seeker = (min(self.seeker[0] + 1, 4), self.seeker[1])
        elif action == 1:  # move left
            self.seeker = (self.seeker[0], max(self.seeker[1] - 1, 0))
        elif action == 2:  # move up
            self.seeker = (max(self.seeker[0] - 1, 0), self.seeker[1])
        elif action == 3:  # move right
            self.seeker = (self.seeker[0], min(self.seeker[1] + 1, 4))
        else:
            raise ValueError("Invalid action")

        obs = self.get_observation()
        rew = self.get_reward()
        done = self.is_done()
        return obs, rew, done, self.info  # returns the observation, reward, whether we're done and any additional information
    
    def render(self, *args, **kwargs):
        """Render the environment, e.g. by printing its representations"""
        os.system('cls' if os.name == 'nt' else 'clear')  # clear the screen

        grid = [['| ' for _ in range(5)] + ["|\n"] for _ in range(5)]
        grid[self.goal[0]][self.goal[1]] = '|G'
        grid[self.seeker[0]][self.seeker[1]] = '|S'  # draw the grid and mark the goal as G and the seeker as S on it
        print(''.join([''.join(grid_row) for grid_row in grid]))  # the grid then gets rendered by printing it to your screen

In [8]:
import time

environment = Environment()

while not environment.is_done():
    random_action = environment.action_space.sample()
    environment.step(random_action)
    time.sleep(0.1)
    environment.render()

| | | | | |
|S| | | | |
| | | | | |
| | | | | |
| | | | |G|

| | | | | |
|S| | | | |
| | | | | |
| | | | | |
| | | | |G|

| | | | | |
| | | | | |
|S| | | | |
| | | | | |
| | | | |G|

| | | | | |
| | | | | |
|S| | | | |
| | | | | |
| | | | |G|

| | | | | |
| | | | | |
| | | | | |
|S| | | | |
| | | | |G|

| | | | | |
| | | | | |
| | | | | |
|S| | | | |
| | | | |G|

| | | | | |
| | | | | |
| | | | | |
|S| | | | |
| | | | |G|

| | | | | |
| | | | | |
| | | | | |
|S| | | | |
| | | | |G|

| | | | | |
| | | | | |
|S| | | | |
| | | | | |
| | | | |G|

| | | | | |
| | | | | |
| | | | | |
|S| | | | |
| | | | |G|

| | | | | |
| | | | | |
| | | | | |
| |S| | | |
| | | | |G|

| | | | | |
| | | | | |
| | | | | |
| | | | | |
| |S| | |G|

| | | | | |
| | | | | |
| | | | | |
| | | | | |
| |S| | |G|

| | | | | |
| | | | | |
| | | | | |
| | | | | |
|S| | | |G|

| | | | | |
| | | | | |
| | | | | |
|S| | | | |
| | | | |G|

| | | | | |
| | | | | |
|S| | | | |
| | | | | |
| | | | |G|

| | | | | |
| | | | | |


In [9]:
import numpy as np


class Policy:
    def __init__(self, env):
        """
        A policy suggests actions based on the current state. 
        We do this by tracking the value of each state-action pair.
        """
        self.state_action_table = [
            [0 for _ in range(env.action_space.n)]
            for _ in range(env.observation_space.n)  # define a nested list of values for each state-action pair, initialised to zero
        ]
        self.action_space = env.action_space
    
    def get_action(self, state, explore=True, epsilon=0.1):  # explore random actions on demand so that we don't get stuck in suboptimal behaviour
        """Explore randomly or exploit the best value currently available"""
        if explore and random.uniform(0, 1) < epsilon:
            return self.action_space.sample()
        return np.argmax(self.state_action_table[state])  # return the action with the highest value in the lookup table, given the current state

In [19]:
class Simulation(object):
    def __init__(self, env):
        """Simulates rollouts of an environment, given a policy to follow"""
        self.env = env
    
    def rollout(self, policy, render=False, explore=True, epsilon=0.1):
        """Returns experiences for a policy rollout"""
        experiences = []
        state  = self.env.reset()
        done = False
        while not done:
            action = policy.get_action(state, explore, epsilon)
            next_state, reward, done, info = self.env.step(action)
            experiences.append([state, action, reward, next_state])
            state = next_state
            if render:
                time.sleep(0.05)
                self.env.render()
        return experiences

In [20]:
untrained_policy = Policy(environment)
sim = Simulation(environment)

exp = sim.rollout(untrained_policy, render=True, epsilon=1.0)  # rollout one full game with an "untrained" policy that we render
for row in untrained_policy.state_action_table:
    print(row)  # the state-action values are currently all zero

| | | | | |
|S| | | | |
| | | | | |
| | | | | |
| | | | |G|

| | | | | |
| | | | | |
|S| | | | |
| | | | | |
| | | | |G|

| | | | | |
| | | | | |
| |S| | | |
| | | | | |
| | | | |G|

| | | | | |
| | | | | |
| | | | | |
| |S| | | |
| | | | |G|

| | | | | |
| | | | | |
| | | | | |
|S| | | | |
| | | | |G|

| | | | | |
| | | | | |
| | | | | |
| | | | | |
|S| | | |G|

| | | | | |
| | | | | |
| | | | | |
|S| | | | |
| | | | |G|

| | | | | |
| | | | | |
| | | | | |
|S| | | | |
| | | | |G|

| | | | | |
| | | | | |
|S| | | | |
| | | | | |
| | | | |G|

| | | | | |
| | | | | |
| | | | | |
|S| | | | |
| | | | |G|

| | | | | |
| | | | | |
| | | | | |
| | | | | |
|S| | | |G|

| | | | | |
| | | | | |
| | | | | |
|S| | | | |
| | | | |G|

| | | | | |
| | | | | |
| | | | | |
|S| | | | |
| | | | |G|

| | | | | |
| | | | | |
| | | | | |
| | | | | |
|S| | | |G|

| | | | | |
| | | | | |
| | | | | |
| | | | | |
|S| | | |G|

| | | | | |
| | | | | |
| | | | | |
| | | | | |
|S| | | |G|

| | | | | |
| | | | | |


In [21]:
def update_policy(policy, experiences, weight=0.1, discount_factor=0.9):
    """Updates a given policy with a list of (state, action, reward, state) experiences"""
    for state, action, reward, next_state in experiences:  # loop through all experiences in order
        next_max = np.max(policy.state_action_table[next_state])  # choose the maximum value among all possible actions in the next_state
        value = policy.state_action_table[state][action]  # extract the current state-action value
        new_value = (1 - weight) * value + weight * (reward + discount_factor * next_max)  # the new value is the weighted sum of the old value and expected value
        policy.state_action_table[state][action] = new_value  # after updating, set the new state_action_table value

In [22]:
def train_policy(env, num_episodes=10000, weight=0.1, discount_factor=0.9):
    """Training a policy by updating it with rollout experiences"""
    policy = Policy(env)
    sim = Simulation(env)
    for _ in range(num_episodes):
        experiences = sim.rollout(policy)  # collect experiences for each game
        update_policy(policy, experiences, weight, discount_factor)  # update our policy with those experiences
    return policy

trained_policy = train_policy(environment)  # train and return a policy for our environment

In [23]:
def evaluate_policy(env, policy, num_episodes=10):
    """Evaluate a trained policy through rollouts"""
    simulation = Simulation(env)
    steps = 0

    for _ in range(num_episodes):
        experiences = simulation.rollout(policy, render=True, explore=False)  # set explore to False to fully exploit the trained policy's learnings
        steps += len(experiences)  # length of experiences in the number of steps we took to finish the game
    
    print(f"{steps / num_episodes} steps on average for a total of {num_episodes} episodes.")
    return steps / num_episodes

evaluate_policy(environment, trained_policy)

| | | | | |
|S| | | | |
| | | | | |
| | | | | |
| | | | |G|

| | | | | |
| | | | | |
|S| | | | |
| | | | | |
| | | | |G|

| | | | | |
| | | | | |
| | | | | |
|S| | | | |
| | | | |G|

| | | | | |
| | | | | |
| | | | | |
| | | | | |
|S| | | |G|

| | | | | |
| | | | | |
| | | | | |
| | | | | |
| |S| | |G|

| | | | | |
| | | | | |
| | | | | |
| | | | | |
| | |S| |G|

| | | | | |
| | | | | |
| | | | | |
| | | | | |
| | | |S|G|

| | | | | |
| | | | | |
| | | | | |
| | | | | |
| | | | |S|

| | | | | |
|S| | | | |
| | | | | |
| | | | | |
| | | | |G|

| | | | | |
| | | | | |
|S| | | | |
| | | | | |
| | | | |G|

| | | | | |
| | | | | |
| | | | | |
|S| | | | |
| | | | |G|

| | | | | |
| | | | | |
| | | | | |
| | | | | |
|S| | | |G|

| | | | | |
| | | | | |
| | | | | |
| | | | | |
| |S| | |G|

| | | | | |
| | | | | |
| | | | | |
| | | | | |
| | |S| |G|

| | | | | |
| | | | | |
| | | | | |
| | | | | |
| | | |S|G|

| | | | | |
| | | | | |
| | | | | |
| | | | | |
| | | | |S|

| | | | | |
|S| | | | |


8.0

In [26]:
import ray

ray.init()

@ray.remote
class SimulationActor(Simulation):  # this Ray actor wraps our Simulation class in a straightforward way
    """Ray actor for a Simulation"""
    def __init__(self):
        env = Environment()
        super().__init__(env)

In [28]:
def train_policy_parallel(env, num_episodes=1000, num_simulations=4):
    """Parallel policy training function"""
    policy = Policy(env)  # initialise a policy for the given environment
    simulations = [SimulationActor.remote() for _ in range(num_simulations)]  # instead of one simulation, create four simulation actors

    policy_ref = ray.put(policy)  # put the policy into the object store
    for _ in range(num_episodes):
        experiences = [sim.rollout.remote(policy_ref) for sim in simulations]  # for each of the 1000 episodes, collect experience data in parallel using our simulation actors

        while len(experiences) > 0:
            finished, experiences = ray.wait(experiences)  # finished rollouts can be retrieved from the object store and used to update the policy
            for xp in ray.get(finished):
                update_policy(policy, xp)
    return policy

In [29]:
parallel_policy = train_policy_parallel(environment)
evaluate_policy(environment, parallel_policy)

| | | | | |
|S| | | | |
| | | | | |
| | | | | |
| | | | |G|

| | | | | |
| | | | | |
|S| | | | |
| | | | | |
| | | | |G|

| | | | | |
| | | | | |
| | | | | |
|S| | | | |
| | | | |G|

| | | | | |
| | | | | |
| | | | | |
| | | | | |
|S| | | |G|

| | | | | |
| | | | | |
| | | | | |
| | | | | |
| |S| | |G|

| | | | | |
| | | | | |
| | | | | |
| | | | | |
| | |S| |G|

| | | | | |
| | | | | |
| | | | | |
| | | | | |
| | | |S|G|

| | | | | |
| | | | | |
| | | | | |
| | | | | |
| | | | |S|

| | | | | |
|S| | | | |
| | | | | |
| | | | | |
| | | | |G|

| | | | | |
| | | | | |
|S| | | | |
| | | | | |
| | | | |G|

| | | | | |
| | | | | |
| | | | | |
|S| | | | |
| | | | |G|

| | | | | |
| | | | | |
| | | | | |
| | | | | |
|S| | | |G|

| | | | | |
| | | | | |
| | | | | |
| | | | | |
| |S| | |G|

| | | | | |
| | | | | |
| | | | | |
| | | | | |
| | |S| |G|

| | | | | |
| | | | | |
| | | | | |
| | | | | |
| | | |S|G|

| | | | | |
| | | | | |
| | | | | |
| | | | | |
| | | | |S|

| | | | | |
|S| | | | |


8.0